In [53]:
import sys
sys.path.append('../../')
import general_utils as gu
import img_utils as iu

In [54]:
from collections import namedtuple
from net import *
from net.downsampler import *
from net.losses import StdLoss, GradientLoss, ExtendedL1Loss, GrayLoss
from net.losses import ExclusionLoss, TVLoss
from net.noise import get_noise
from PIL import Image
from pillow_heif import register_heif_opener
register_heif_opener()

import numpy as np
import math
import torch
import cv2
from torchvision import transforms
import time
from pathlib import Path
from tqdm.notebook import tqdm



torch.manual_seed(0)

def scale_fill(im, target_width, target_height, return_offset=False):
    '''
    Resize PIL image keeping ratio and adding zero-valued background.
    '''
    target_ratio = target_height / target_width
    im_ratio = im.height / im.width
    if target_ratio > im_ratio:
        # It must be fixed by width
        resize_width = target_width
        resize_height = round(resize_width * im_ratio)
    else:
        # Fixed by height
        resize_height = target_height
        resize_width = round(resize_height / im_ratio)

    image_resize = im.resize((resize_width, resize_height), Image.ANTIALIAS)
    background = Image.new('RGBA', (target_width, target_height), 0)
    offset = (round((target_width - resize_width) / 2), round((target_height - resize_height) / 2))
    background.paste(image_resize, offset)

    if return_offset:
        return background.convert('RGB'), offset, image_resize.size 

    return background.convert('RGB')

def reverse_scale_fill(im, original_size, offset, image_resize_size):
    '''
    original_size: (W, H)
    image_resize_size: (W, H) : size of the original image in the scalefill image 
    offset: (W_dim, H_dim)
    Example: scalefill from 720, 960 to 960, 960 will have offset=120,0 and image_resize_size=720,960
    '''
    box = offset[0], offset[1], offset[0]+image_resize_size[0], offset[1]+image_resize_size[1]
    return im.crop(box).resize(original_size)

EnhancementResult = namedtuple("EnhancementResult", ['reflection', 'illumination'])

class Enhancement(object):
    def __init__(self, image_name, image, target_size=960, plot_during_training=False, show_every=10, num_iter=300):
        self.original_image = image
        self.size = image.size # (height, width)
        self.target_size = target_size
        self.image_np = None
        self.images_torch = None
        self.plot_during_training = plot_during_training
        # self.ratio = ratio
        self.psnrs = []
        self.show_every = show_every
        self.image_name = image_name
        self.num_iter = num_iter
        self.loss_function = None
        # self.ratio_net = None
        self.parameters = None
        self.learning_rate = 0.01
        self.input_depth = 3  # This value could affect the performance. 3 is ok for natural image, if your
                            #images are extremely dark, you may consider 8 for the value.
        self.data_type = torch.FloatTensor
        self.reflection_net_inputs = None
        self.illumination_net_inputs = None
        self.original_illumination = None
        self.original_reflection = None
        self.reflection_net = None
        self.illumination_net = None
        self.total_loss = None
        self.reflection_out = None
        self.illumination_out = None
        self.current_result = None
        self.best_result = None
        self._init_all()

        print('Original image size:', self.size)

    def _init_all(self):
        self._init_images()
        self._init_decomposition()
        self._init_nets()
        self._init_inputs()
        self._init_parameters()
        self._init_losses()


    def _maxRGB(self):
        '''
        self.image: pil image, input low-light image
        :return: np, initial illumnation
        '''
        (R, G, B) = self.image.split()
        I_0 = np.array(np.maximum(np.maximum(R, G), B))
        return I_0

    def _init_decomposition(self):
        temp = self._maxRGB() # numpy
        # get initial illumination map
        #IMPORTANT: min clip value is a hyper param
        self.original_illumination = np.clip(np.asarray([temp for _ in range(3)]), 2, 255) #range [0,255]
        # get initial reflection
        self.original_reflection = self.image_np / self.original_illumination #range [0, 255] / [0,255] = [0, 1]

        self.original_illumination = np_to_torch(self.original_illumination).type(self.data_type)
        self.original_reflection = np_to_torch(np.asarray(self.original_reflection)).type(self.data_type)

    def _init_images(self):

        self.original_image_torch = transforms.PILToTensor()(self.original_image)
        self.image, self.scalefill_offset, self.image_resize_size = scale_fill(self.original_image, self.target_size, self.target_size, return_offset=True)
        # self.image = transforms.Resize((512, 512))(self.original_image)
        self.image_np = pil_to_np(self.image)  # pil image to numpy
        self.image_torch = np_to_torch(self.image_np).type(self.data_type)

    def _init_inputs(self):
        if self.image_torch is not None:
            size = (self.image_torch.shape[2], self.image_torch.shape[3])
            # print(size)
        input_type = 'noise'
        # input_type = 'meshgrid'
        self.reflection_net_input = get_noise(self.input_depth,
                                                  input_type, size).type(self.data_type).detach()
        self.illumination_net_input = get_noise(self.input_depth,
                                             input_type, size).type(self.data_type).detach()


    def _init_parameters(self):
        self.parameters = [p for p in self.reflection_net.parameters()] + \
                          [p for p in self.illumination_net.parameters()]
        self.optimizer = torch.optim.Adam(self.parameters, lr=self.learning_rate)


    def weight_init(self, m):
        classname = m.__class__.__name__
        if classname.find('Conv') != -1:
            n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            m.weight.data.normal_(0.0, 0.5 * math.sqrt(2. / n))
            if m.bias is not None:
                m.bias.data.zero_()
        elif classname.find('BatchNorm') != -1:
            m.weight.data.fill_(1)
            m.bias.data.zero_()
        elif classname.find('Linear') != -1:
            n = m.weight.size(1)
            m.weight.data.normal_(0, 0.01)
            m.bias.data = torch.ones(m.bias.data.size())

    def _init_nets(self):
        pad = 'zero'
        self.reflection_net = skip(self.input_depth, 3,
               num_channels_down = [8, 16, 32, 64, 128, 256],
               num_channels_up   = [8, 16, 32, 64, 128, 256],
               num_channels_skip = [0, 0, 0, 0, 0, 0],
               filter_size_down = 3, filter_size_up = 3, filter_skip_size=1,
               upsample_mode='bilinear',
               downsample_mode='avg',
               need_sigmoid=True, need_bias=True, pad=pad)
        self.reflection_net.apply(self.weight_init).type(self.data_type)


        self.illumination_net = skip(self.input_depth, 3,
               num_channels_down = [8, 16, 32, 64, 128, 256],
               num_channels_up   = [8, 16, 32, 64, 128, 256],
               num_channels_skip = [0, 0, 0, 0, 0, 0],
               filter_size_down = 3, filter_size_up = 3, filter_skip_size=1,
               upsample_mode='bilinear',
               downsample_mode='avg',
               need_sigmoid=True, need_bias=True, pad=pad)
        self.illumination_net.apply(self.weight_init).type(self.data_type)



    def _init_losses(self):
        self.l1_loss = nn.SmoothL1Loss().type(self.data_type) # for illumination
        self.mse_loss = nn.MSELoss().type(self.data_type)     # for reflection and reconstruction
        self.exclusion_loss =  ExclusionLoss().type(self.data_type)
        self.tv_loss = TVLoss().type(self.data_type)
        self.gradient_loss = GradientLoss().type(self.data_type)



    def optimize(self):
        # torch.backends.cudnn.enabled = True
        # torch.backends.cudnn.benchmark = True
        # optimizer = SGLD(self.parameters, lr=self.learning_rate)
        
        start = time.time()
        for j in tqdm(range(self.num_iter)):
            self.optimizer.zero_grad()
            self.calculate_loss()
            if self.plot_during_training:
                self._save_result(j)
            self.optimizer.step()
        end = time.time()
        print("time:%.4f"%(end-start))
        self.best_result.save(str(self.image_name))

    def calculate_loss(self):

        # reg_noise_std = 1 / 10000.

        illumination_net_input = self.illumination_net_input #+ \
                                #  self.illumination_net_input.clone().normal_(mean=0, std=reg_noise_std)
        reflection_net_input = self.reflection_net_input #+ \
                            #    self.reflection_net_input.clone().normal_(mean=0, std=reg_noise_std)


        self.illumination_out = self.illumination_net(illumination_net_input)
        self.reflection_out = self.reflection_net(reflection_net_input)

        # weighted with the gradient of latent reflectance
        self.total_loss = 0.5*self.tv_loss(self.illumination_out, self.reflection_out)
        self.total_loss += 0.0001*self.tv_loss(self.reflection_out)
        self.total_loss += self.l1_loss(self.illumination_out, self.original_illumination/255)
        self.total_loss += self.mse_loss(
            self.illumination_out*self.reflection_out, 
            self.image_torch/255
        )
        self.total_loss.backward()


    def _obtain_current_result(self, step):
        """
        puts in self.current result the current result.
        also updates the best result
        :return:
        """
        if step == self.num_iter - 1 or step % 8 == 0:
            reflection_out_np = np.clip(torch_to_np(self.reflection_out),0,1)
            illumination_out_np = np.clip(torch_to_np(self.illumination_out),0,1)
            # psnr = compare_psnr(np.clip(self.image_np,0,1),  reflection_out_np * illumination_out_np)
            # self.psnrs.append(psnr)

            self.current_result = EnhancementResult(reflection=reflection_out_np, illumination=illumination_out_np)
            # if self.best_result is None or self.best_result.psnr < self.current_result.psnr:
            #     self.best_result = self.current_result

    def _save_result(self, step):
        if (step % self.show_every == self.show_every - 1) or (step==0) or (step==self.num_iter-1):
            print('Iteration {:5d}    Loss {:5f}'.format(step,self.total_loss.item()))

            self.get_enhanced(step)

    def gamma_trans(self, img, gamma):
        gamma_table = [np.power(x / 255.0, gamma) * 255.0 for x in range(256)]
        gamma_table = np.round(np.array(gamma_table)).astype(np.uint8)
        return cv2.LUT(img, gamma_table)

    def adjust_gammma(self,img):
        image_gamma_correct = self.gamma_trans(img, 0.5)
        return image_gamma_correct

    def get_enhanced(self, step, flag=False):
        (R, G, B) = self.image.split()
        ini_illumination = self.illumination_out * 255.0
        ini_illumination = torch_to_np(ini_illumination).transpose(1, 2, 0) # H x W x C

        # If the input image is extremely dark, setting the flag as True can produce promising result.
        if flag==True:
            ini_illumination = np.clip(np.max(ini_illumination, axis=2), 0.0000002, 255)
        else:
            ini_illumination = np.clip(self.adjust_gammma(ini_illumination.astype(np.uint8)), 0.0000002, 255)
        
        R = R / ini_illumination[:,:,0] # R range [0,255], ini_illu supposedly range [0, 255]
        G = G / ini_illumination[:,:,1]
        B = B / ini_illumination[:,:,2]
        self.best_result = Image.fromarray(np.clip(cv2.merge([R, G, B])*255, 0.02, 255).astype(np.uint8), mode='RGB')#.resize(self.size)    
        self.best_result = reverse_scale_fill(self.best_result, self.size, self.scalefill_offset, self.image_resize_size)
        self.best_result.save('output/'+self.image_name.stem+('_{}.png'.format(step)))




In [ ]:


img_folder = Path('/Users/thong/Desktop/image-enhancement/original_images/5-4-2022')
output_path = gu.folder(Path('./result/'+img_folder.parts[-1]))
print('Output path = ', output_path)
img_path_list = iu.get_image_paths(img_folder)
print(img_path_list)

for img_path in img_path_list:
    print('-------------Processing '+img_path.name)
    image = Image.open(img_path)
    s = Enhancement(output_path/img_path.name.replace('original', 'best'), 
                    image, 
                    target_size=960,
                    plot_during_training=True, show_every=50, num_iter=200)
    s.optimize()
    

    


# Past stuff

In [9]:
# parser = argparse.ArgumentParser()
# parser.add_argument("--input", "-i", type=str, default='data/Test', help='test image folder')
# parser.add_argument("--result", "-r", type=str, default='./result', help='result folder')
# arg = parser.parse_args()


img_path = Path('/Users/thong/Desktop/image-enhancement/original_20-3-2022_1.jpeg')
output_path = Path('./result')
output_path.mkdir(parents=True, exist_ok=True)

img = Image.open(img_path).convert('RGB') #LOLdataset/eval15/low/1.png


In [ ]:
img = img.transpose(Image.ROTATE_270)
img

In [72]:

s = Enhancement(output_path/img_path.name.replace('original', 'best'), img, plot_during_training=True, show_every=10, num_iter=200)


Original image size: (3456, 4608)


In [ ]:
s.optimize()